In [ ]:
import requests
import json
import os
from tqdm import tqdm
from datetime import datetime
from fpdf import FPDF

# URL base de la API de GW2 (con idioma español)
API_URL = "https://api.guildwars2.com/v2/commerce/prices"
ITEMS_URL = "https://api.guildwars2.com/v2/items"
IDS_URL = "https://api.guildwars2.com/v2/commerce/prices"
LANG_HEADER = {"Accept-Language": "es"}

NAMES_CACHE_FILE = "item_names_cache.json"
IGNORED_IDS_FILE = "ignored_ids.json"
VALUABLE_IDS_FILE = "valuable_ids.json"
RESULTS_FOLDER = "resultados"
MAX_TOTAL_INVESTMENT_PERCENTAGE = 0.2  # 20% del total disponible
PRICE_DISPARITY_LIMIT = 3  # Máxima diferencia permitida entre precio de venta y compra

# Función para convertir oro a cobre
def convert_gold_to_copper(gold):
    return gold * 10000

# Funciones de caché

def load_json_file(filename):
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)
    return {}

def save_json_file(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# Función para obtener todos los IDs de ítems disponibles en el comercio
def get_all_item_ids():
    response = requests.get(IDS_URL)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error al obtener lista de IDs de la API")
        return []

# Función para obtener nombres faltantes de ítems (en español)
def update_item_names_cache(item_ids, cache, ignored_ids):
    unknown_ids = [i for i in item_ids if str(i) not in cache and i not in ignored_ids]
    for i in tqdm(range(0, len(unknown_ids), 200), desc="Actualizando nombres"):
        chunk = unknown_ids[i:i + 200]
        response = requests.get(f"{ITEMS_URL}?ids={','.join(map(str, chunk))}", headers=LANG_HEADER)
        if response.status_code == 200:
            items_data = response.json()
            for item in items_data:
                name = item.get('name', 'Desconocido')
                if name != 'Desconocido':
                    cache[str(item['id'])] = name
                else:
                    ignored_ids.add(item['id'])
        else:
            print(f"Error al obtener nombres para IDs {chunk}")
            for item_id in chunk:
                ignored_ids.add(item_id)
    return cache, ignored_ids

# Introducir fondos en solo oro
gold = int(input("Introduce la cantidad total de oro disponible: "))
TOTAL_FUNDS = convert_gold_to_copper(gold)

# Preguntar si se desea ejecutar en modo completo (ignorar caché de valiosos)
modo_completo = input("¿Deseas ejecutar en modo completo? (s/n): ").strip().lower() == 's'

# Porcentajes
INVESTMENT_PERCENTAGE = 0.01  # 1% por ítem
MAX_INVESTMENT = TOTAL_FUNDS * INVESTMENT_PERCENTAGE
VALUABLE_THRESHOLD = TOTAL_FUNDS * 0.02
MAX_TOTAL_INVESTMENT = TOTAL_FUNDS * MAX_TOTAL_INVESTMENT_PERCENTAGE

# Obtener precios de ítems
def get_item_prices(item_ids):
    prices = []
    for i in tqdm(range(0, len(item_ids), 200), desc="Obteniendo precios"):
        chunk = item_ids[i:i + 200]
        response = requests.get(f"{API_URL}?ids={','.join(map(str, chunk))}")
        if response.status_code == 200:
            prices.extend(response.json())
        else:
            print(f"Error al obtener precios para IDs {chunk}")
    return prices

# Analizar oportunidades y detectar valiosos
def analyze_flipping(prices, max_investment, threshold):
    opportunities = []
    valuable_ids = set()
    for item in prices:
        sell_price = item['sells']['unit_price']  # precio al que tú compras instantáneamente
        buy_price = item['buys']['unit_price']    # precio al que tú vendes instantáneamente
        sell_quantity = item['sells']['quantity']
        buy_quantity = item['buys']['quantity']

        if sell_price == 0 or buy_price == 0 or sell_price > buy_price * PRICE_DISPARITY_LIMIT:
            continue

        if buy_price > threshold:
            valuable_ids.add(item['id'])

        profit = buy_price * 0.85 - sell_price
        if profit > 0 and sell_price <= max_investment:
            max_quantity = max_investment // sell_price
            opportunities.append({
                "item_id": item['id'],
                "buy_price": sell_price,  # compras al sell_price (instantáneo)
                "sell_price": buy_price,  # vendes al buy_price (instantáneo)
                "profit": profit,
                "max_quantity": max_quantity,
                "total_cost": sell_price * max_quantity,
                "buy_quantity": buy_quantity,
                "sell_quantity": sell_quantity
            })
    return sorted(opportunities, key=lambda x: x['profit'], reverse=True), valuable_ids

# Lógica principal
if __name__ == "__main__":
    item_names_cache = load_json_file(NAMES_CACHE_FILE)
    ignored_ids = set(load_json_file(IGNORED_IDS_FILE))
    valuable_ids_cache = set(load_json_file(VALUABLE_IDS_FILE))

    all_item_ids = get_all_item_ids()
    item_ids_to_check = all_item_ids if modo_completo or not valuable_ids_cache else list(valuable_ids_cache)
    prices = get_item_prices(item_ids_to_check)
    item_names_cache, ignored_ids = update_item_names_cache([item['id'] for item in prices], item_names_cache, ignored_ids)
    save_json_file(item_names_cache, NAMES_CACHE_FILE)
    save_json_file(list(ignored_ids), IGNORED_IDS_FILE)

    flipping_opportunities, new_valuable_ids = analyze_flipping(prices, MAX_INVESTMENT, VALUABLE_THRESHOLD)
    valuable_ids_cache.update(new_valuable_ids)
    save_json_file(list(valuable_ids_cache), VALUABLE_IDS_FILE)

    # Filtrar oportunidades hasta el 20% del capital total y excluir desconocidos
    filtered_opps = []
    total_used = 0
    for opp in flipping_opportunities:
        item_name = item_names_cache.get(str(opp['item_id']), 'Desconocido')
        if item_name == 'Desconocido':
            ignored_ids.add(opp['item_id'])
            continue
        if total_used + opp['total_cost'] > MAX_TOTAL_INVESTMENT:
            break
        opp['item_name'] = item_name
        filtered_opps.append(opp)
        total_used += opp['total_cost']

    save_json_file(list(ignored_ids), IGNORED_IDS_FILE)

    print("\nMejores Oportunidades de Flipping:")
    print(f"Total invertido: {total_used}c de {int(MAX_TOTAL_INVESTMENT)}c (máximo permitido)\n")
    print(f"{'Nombre':40} | {'Comprar':>8} | {'Vender':>8} | {'Ganancia':>8} | {'Cantidad':>8} | {'Total':>8} | {'BuyQty':>7} | {'SellQty':>7}")
    print("-" * 120)
    for opp in filtered_opps:
        print(f"{opp['item_name'][:40]:40} | {opp['buy_price']:>8}c | {opp['sell_price']:>8}c | {opp['profit']:>8.0f}c | {opp['max_quantity']:>8} | {opp['total_cost']:>8}c | {opp['buy_quantity']:>7} | {opp['sell_quantity']:>7}")

    os.makedirs(RESULTS_FOLDER, exist_ok=True)
    now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    pdf_path = os.path.join(RESULTS_FOLDER, f"flipping_{now}.pdf")

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=10)
    pdf.cell(200, 10, txt="Mejores Oportunidades de Flipping - GW2", ln=True, align='C')
    pdf.ln(5)
    pdf.cell(200, 10, txt=f"Total invertido: {total_used}c de {int(MAX_TOTAL_INVESTMENT)}c", ln=True)
    pdf.ln(5)

    pdf.set_font("Arial", style='B', size=8)
    pdf.cell(60, 8, "Nombre", border=1)
    pdf.cell(20, 8, "Comprar", border=1)
    pdf.cell(20, 8, "Vender", border=1)
    pdf.cell(20, 8, "Ganancia", border=1)
    pdf.cell(20, 8, "Cantidad", border=1)
    pdf.cell(20, 8, "Total", border=1)
    pdf.cell(15, 8, "BuyQty", border=1)
    pdf.cell(15, 8, "SellQty", border=1)
    pdf.ln()

    pdf.set_font("Arial", size=7)
    for opp in filtered_opps:
        pdf.cell(60, 8, opp['item_name'][:50], border=1)
        pdf.cell(20, 8, f"{opp['buy_price']}c", border=1)
        pdf.cell(20, 8, f"{opp['sell_price']}c", border=1)
        pdf.cell(20, 8, f"{int(opp['profit'])}c", border=1)
        pdf.cell(20, 8, str(opp['max_quantity']), border=1)
        pdf.cell(20, 8, f"{opp['total_cost']}c", border=1)
        pdf.cell(15, 8, str(opp['buy_quantity']), border=1)
        pdf.cell(15, 8, str(opp['sell_quantity']), border=1)
        pdf.ln()

    pdf.output(pdf_path)
    print(f"\n📄 Resultados exportados en: {pdf_path}")


Obteniendo precios:  76%|███████▌  | 37/49 [00:11<00:03,  3.37it/s]